# Keys generation

Intro
- https://github.com/pcaversaccio/mnemonic-to-private-key


Concepts:
- key pair = the basic block needed by crypto operations
  - BUT we want many KPs for privacy! Ex. not showing big amounts, all history etc.
- wallet = a mechanism to hold a group of key pairs (and generate ad-hoc)
- wallet tree = tree of derived KP = "Hierarchical Deterministic Wallet structure"
  - from a single "seed" (root)
  - into "leaves" KP
  - "derived" along a branch = "derivation path"
    - ⚠️⚠️⚠️ the derivation alogrithm depends on the intended usage of the KPs! (crypto primitives that will be used)
    - "levels" = "segment path"
    - "hardened" vs "public" derivation = ???
      - "It’s easy to shoot yourself in foot with non-hardened keys, which could allow simple de-anonimization of all addresses" ???
- [PBKDF2](https://en.wikipedia.org/wiki/PBKDF2) = a key derivation algo
- [BIP39 Mnemonic code for generating deterministic keys](https://github.com/bitcoin/bips/blob/master/bip-0039.mediawiki)
  * mnemonic sentence = a group of easy to remember words
  * A user may decide to protect their mnemonic with a passphrase
  * to be used as the wallet root
  * +++ [Mnemonic Code Converter](https://iancoleman.io/bip39/)
- [BIP32 Hierarchical Deterministic Wallets](https://github.com/bitcoin/bips/blob/master/bip-0032.mediawiki)
  - ⚠️⚠️⚠️ [Warnings about BIP32](https://github.com/paulmillr/scure-bip32?tab=readme-ov-file#warnings-about-bip32)
    - competitor? https://eips.ethereum.org/EIPS/eip-2333
  - algorithm for a derivation path FOR BITCOIN (doesn't work for all chains, depends on the crypto primitives)
  - pretty advanced
  - "private extended key"
  - "serialization format" = ???
  - "selective sharing" = pruning a branch? sharing only public keys?
  - complex technical demo at [bip32.org](http://bip32.org/)
  - ??? "n order to prevent the childkeys from depending solely on the parent key itself, we extend both private and public keys first with an extra   256 bits of entropy. This extension, called the chain code, is identical for corresponding private and public keys, and consists of 32 bytes."
  - "hardened" = derived + 2^31 and different algo?
  - "xprv" or "xpub" on mainnet, "tprv" or "tpub" on testnet
  - ⚠️ "invalid keys" = "BIP32 is unusable for many different elliptic curves.
    For example, ETH2 uses bls12-381 curve, and with bip32 54% of generated keys would be invalid.
    So, they’re using much better BLS-only EIP-2333 as a replacement."
- [BIP43 Purpose Field for Deterministic Wallets](https://github.com/bitcoin/bips/blob/master/bip-0043.mediawiki)
  - first "level"/"segment" of the derivation path
- [BIP44 Multi-Account Hierarchy for Deterministic Wallets](https://github.com/bitcoin/bips/blob/master/bip-0044.mediawiki)
  - `m / purpose' / coin_type' / account' / change / address_index`
- [BIP49 Derivation scheme for P2WPKH-nested-in-P2SH based accounts](https://github.com/bitcoin/bips/blob/master/bip-0049.mediawiki)
- [BIP85 Deterministic Entropy From BIP32 Keychains](https://github.com/bitcoin/bips/blob/master/bip-0085.mediawiki)
- entropy???
- pub/prv

TODO
- https://github.com/pcaversaccio/mnemonic-to-private-key
- weak PK2 https://gnusha.org/pi/bitcoindev/5797C3A7.5030600@jonasschnelli.ch/
https://iancoleman.io/bip39/

## Root = Mnemonic Phrase / seed phrase

generation using [`@scure/bip39`](https://github.com/paulmillr/scure-bip39)

> Since the vast majority of BIP39 wallets supports only the English wordlist, it is strongly discouraged to use non-English wordlists for generating the mnemonic sentences.
>
> (BIP39)

In [ ]:
import * as bip39 from "npm:@scure/bip39"
import { wordlist } from "npm:@scure/bip39/wordlists/english.js"
const MAX_SUPPORTED_ENTROPY = 256
const SEED_PHRASE = bip39.generateMnemonic(wordlist, MAX_SUPPORTED_ENTROPY)

In [ ]:
import { strict as assert } from "node:assert"
console.log(SEED_PHRASE)
assert(bip39.validateMnemonic(SEED_PHRASE, wordlist))

## seed = mnemonic + passphrase + entropy

### seed phrase

- [BIP39 Mnemonic code for generating deterministic keys](https://github.com/bitcoin/bips/blob/master/bip-0039.mediawiki)
  * to create a binary seed from the mnemonic, we use the PBKDF2 function with a mnemonic sentence (in UTF-8 NFKD) used as the password and the string "mnemonic" + passphrase (again in UTF-8 NFKD) used as the salt.
    * The iteration count is set to 2048 and HMAC-SHA512 is used as the pseudo-random function. The length of the derived key is 512 bits (= 64 bytes). 

  ⚠️⚠️⚠️ 2048 is no longer considered good enough!

### passphrase

### entropy


## root node = binary seed?

In [ ]:
const ITERATION_COUNTⵧCOMPATIBLE = 2048 //   ⚠️⚠️⚠️ no longer considered good enough!
const ITERATION_COUNTⵧRECOMMENDED = NaN // ???

In [ ]:
crypto.deriveKey(algorithm, baseKey, derivedKeyType, extractable, keyUsages)

## Derivation

### BIP44

Levels: `m / purpose' / coin_type' / account' / change / address_index`

1. purpose = [BIP43 Purpose Field for Deterministic Wallets](https://github.com/bitcoin/bips/blob/master/bip-0043.mediawiki)
  - `44'` for 'P2PKH or P2SH' [BIP44](https://github.com/bitcoin/bips/blob/master/bip-0044.mediawiki)
  - `49'` for [BIP49](https://github.com/bitcoin/bips/blob/master/bip-0049.mediawiki)
  - `84'` for BIP84 https://github.com/bitcoin/bips/blob/master/bip-0084.mediawiki
  - 10001 to 19999 are reserved for [SLIPs](https://github.com/satoshilabs/slips)
  - `0'` = BIP32 default account (used ?)
1. coin type = [SLIP44](https://github.com/satoshilabs/slips/blob/master/slip-0044.md)
  - This level creates a separate subtree for every cryptocoin, avoiding reusing addresses across cryptocoins and improving privacy issues
  - `0x80000000` = BTC
  - `60'` = `0x8000003c` = ETH
  - `0x800001f5` = SOL
  - `0x80000001` = any testnet (all coins)
1. account = 
  - This level splits the key space into independent user identities, so the wallet never mixes the coins across different accounts.
    Users can use these accounts to organize the funds in the same fashion as bank accounts, ex:
    - for donation purposes (where all addresses are considered public)
    - for saving purposes
    - for common expenses etc. 
1. change
  - ??
1. index
  - any

### others?

https://eips.ethereum.org/EIPS/eip-2333


## Addresses

Let's create a few to interact between themselves

In [ ]:
import { mnemonicToAccount } from "viem/accounts"

const account1 = mnemonicToAccount(SEED_PHRASE)

In [ ]:
// viem = only Ethereum (lock purpose and coin)
import { mnemonicToAccount } from "viem/accounts"
const account2 = mnemonicToAccount(SEED_PHRASE, {
	passphrase: 'passphrase',
	accountIndex: 0,
	changeIndex: 0,
	addressIndex: 0,
})

## Clients